In [1]:
import pip #needed to use the pip functions
for i in pip.get_installed_distributions(local_only=True):print(i)

StatementMeta(marketriskspksm, 73, 3, Finished, Available)

zipp 0.6.0
zict 1.0.0
yfinance 0.1.54
xlwt 1.2.0
XlsxWriter 0.9.6
xlrd 1.0.0
xgboost 0.90
wrapt 1.11.2
widgetsnbextension 2.0.0
wheel 0.30.0
Werkzeug 0.16.1
websocket-client 0.56.0
wcwidth 0.2.5
vega-datasets 0.7.0
urllib3 1.26.2
unicodecsv 0.14.1
typing-extensions 3.7.4
typed-ast 1.4.1
traitlets 4.3.3
tqdm 4.51.0
tornado 6.0.3
torch 1.4.0
toolz 0.10.0
testpath 0.3
terminado 0.6
termcolor 1.1.0
tensorflow 1.14.0
tensorflow-estimator 1.14.0
tensorboard 1.14.0
tblib 1.4.0
tables 3.3.0
sympy 1.0
statsmodels 0.12.1
SQLAlchemy 1.1.9
spyder 3.1.4
sortedcontainers 2.1.0
sortedcollections 0.5.3
snowballstemmer 1.2.1
smart-open 1.8.4
sklearn-pandas 1.7.0
skl2onnx 1.4.9
six 1.15.0
singledispatch 3.4.0.3
simplegeneric 0.8.1
shap 0.34.0
setuptools 50.3.2
SecretStorage 3.1.1
seaborn 0.11.0
scipy 1.5.4
scikit-learn 0.20.3
scikit-image 0.15.0
s3transfer 0.2.1
ruamel.yaml 0.16.10
ruamel.yaml.clib 0.2.2
rope-py3k 0.9.4.post1
retrying 1.3.3
Resource 0.2.1
requests 2.25.0
requests-oauthlib 1.3.0
ratelimi

In [2]:
import urllib.parse, base64
import json
import requests
import pandas as pd
import datetime
import pytz
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

#sentiment_url = 'https://#COGNITIVE_SERVICE_NAME#.api.cognitive.microsoft.com/text/analytics/v3.0/sentiment' # service address 
sentiment_url = 'https://#COGNITIVE_SERVICE_NAME#.cognitiveservices.azure.com/'
api_key = '#COGNITIVE_SERVICE_KEY#'          # Azure Cognitive API Key, replace with your own key


credential = AzureKeyCredential(api_key)
text_analytics_client = TextAnalyticsClient(endpoint=sentiment_url, credential=credential)

StatementMeta(marketriskspksm, 73, 4, Finished, Available)

In [3]:
from azure.storage.blob import (
    BlockBlobService
)

accountName = "#STORAGE_ACCOUNT_NAME#"
accountKey = "#STORAGE_ACCOUNT_KEY#"
containerName = "risk"

blobService = BlockBlobService(account_name=accountName, account_key=accountKey)

StatementMeta(marketriskspksm, 73, 5, Finished, Available)

In [4]:
import datetime
import pytz

currDate = datetime.datetime.now(pytz.timezone('US/Central'))
blobName = "TwitterData/" + str(currDate.year) + "/" + str(currDate.month) + "/" + str(currDate.day - 1)
print(blobName)

   

StatementMeta(marketriskspksm, 73, 6, Finished, Available)

TwitterData/2020/11/17

In [5]:
# Read out CSV file with list of all tickers
nqSdf = spark.read.load('abfss://risk@#DATA_LAKE_NAME#.dfs.core.windows.net/Portfolio/PortfolioCik.csv', 
    format='csv', 
    sep=",",
    header=True)

portfolioData = nqSdf.toPandas()
display(portfolioData)

StatementMeta(marketriskspksm, 73, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, 6e907496-9d5d-4f64-bff5-f182dde8830d)


/opt/spark/python/lib/pyspark.zip/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  'JavaPackage' object is not callable
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.

In [16]:
import glob
import re 

filename_prefix = "/" + blobName 
filename_objects = blobService.list_blobs('risk', prefix = filename_prefix)
#print(filename_objects)

for index, portfolio in portfolioData.iterrows():
   print(portfolio.Ticker)
   if portfolio.Ticker :
      filtered = [fileinfo.name for fileinfo in filename_objects if portfolio.Ticker in fileinfo.name]
      print(filtered)
      consolidatedtweets = pd.DataFrame()

      for tickerfilename in filtered : 
        absolouteFileName = "abfss://risk@#DATA_LAKE_NAME#.dfs.core.windows.net/" + tickerfilename
        print(absolouteFileName)
        nqSdf = spark.read.format('csv') \
                    .option('header',True) \
                    .option('multiLine', True) \
                    .load(absolouteFileName)
        
        consolidatedtweets = consolidatedtweets.append(nqSdf.toPandas())
        scoredTweets = pd.DataFrame()
        consolidatedtweets['Location'] = consolidatedtweets['Location\r'].str.strip(r'\\r').astype(str)
        consolidatedtweets = consolidatedtweets.drop(columns='Location\r')
        
      print("Before drop", consolidatedtweets.shape[0])
      consolidatedtweets = consolidatedtweets.drop_duplicates(subset=['TweetText'])
      print("After Drop", consolidatedtweets.shape[0])

      for index, tweet in consolidatedtweets.iterrows():
            documents = []
            mentions = re.findall("@([a-zA-Z0-9]{1,15})", tweet.TweetText)
            topics = re.findall("#([a-zA-Z0-9]{1,15})", tweet.TweetText)
            documents.append ("'" + tweet.TweetText + "'")
            tweet['mentions'] = mentions
            tweet['topics'] = topics
            tweet['Ticker'] = portfolio.Ticker
            response = text_analytics_client.analyze_sentiment(documents, language="en")
            result = [doc for doc in response if not doc.is_error]
            tweet['sentiment'] = result[0].sentiment
            tweet['positive'] = result[0].confidence_scores.positive
            tweet['negative'] = result[0].confidence_scores.negative
            tweet['neutral'] = result[0].confidence_scores.neutral

            if tweet.Location : 
              tweet.Location = tweet.Location.rstrip('\r')
            else :
              tweet.Location = " "
              tweet.Location = tweet.Location.rstrip('\r')
            
            scoredTweets = scoredTweets.append(tweet,ignore_index=True) 

      output = scoredTweets.to_csv(index=False, header=True, encoding = "utf-8")
      blobService.create_blob_from_text(containerName,  "TwitterData/scored/" + str(currDate.year) + "/" + str(currDate.month) + "/" + str(currDate.day) + "/" + portfolio.Ticker + ".csv", output)
    



      
    
        






           


  



StatementMeta(tester, 10, 18, Finished, Available)

abfss://risk@#DATA_LAKE_NAME#.dfs.core.windows.net/TwitterData/2020/10/28/11/AMZN.csv
abfss://risk@#DATA_LAKE_NAME#.dfs.core.windows.net/TwitterData/2020/10/28/11/ARE.csv
abfss://risk@#DATA_LAKE_NAME#.dfs.core.windows.net/TwitterData/2020/10/28/11/CNP.csv
abfss://risk@#DATA_LAKE_NAME#.dfs.core.windows.net/TwitterData/2020/10/28/11/COST.csv
abfss://risk@#DATA_LAKE_NAME#.dfs.core.windows.net/TwitterData/2020/10/28/11/CRM.csv
abfss://risk@#DATA_LAKE_NAME#.dfs.core.windows.net/TwitterData/2020/10/28/11/CVS.csv
abfss://risk@#DATA_LAKE_NAME#.dfs.core.windows.net/TwitterData/2020/10/28/11/DIS.csv
abfss://risk@#DATA_LAKE_NAME#.dfs.core.windows.net/TwitterData/2020/10/28/11/DPZ.csv
abfss://risk@#DATA_LAKE_NAME#.dfs.core.windows.net/TwitterData/2020/10/28/11/FISV.csv
abfss://risk@#DATA_LAKE_NAME#.dfs.core.windows.net/TwitterData/2020/10/28/11/FMC.csv
abfss://risk@#DATA_LAKE_NAME#.dfs.core.windows.net/TwitterData/2020/10/28/11/GIS.csv
abfss://risk@#DATA_LAKE_NAME#.dfs.core.windows.net/TwitterData

SynapseWidget(Synapse.DataFrame, b5ba4d63-48f5-4464-9798-b450c2c26451)


/opt/spark/python/lib/pyspark.zip/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  'JavaPackage' object is not callable
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
/opt/spark/python/lib/pyspark.zip/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Tried to write record batch with different schema
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.